In [125]:
import pandas as pd
import numpy as np

In [126]:
data = pd.read_excel("URINELOTS.xlsx")
data = data[['UHID','RAMJA Results','AIIMS Result culture']]
data.columns = ['UHID','RAMJA','AIIMS']
data.head()

,UHID,RAMJA,AIIMS
0,20220065432,Enterococcus faecalis,Enterococcus faecalis
1,20240016182,NaN,Contamination
2,20230125389,"E.Coli, K.pneumoniae, P. aeruginosa, A. bauman...",E.coli
3,20180022685,"K.pneumoniae, E. faecalis, P. mirabilis, A. b...",CONTAMINATION
4,20201415102,NaN,NaN


In [127]:
TotalSamples  = data.shape[0]
TotalSamples

146

In [128]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146 entries, 0 to 145
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   UHID    146 non-null    int64 
 1   RAMJA   143 non-null    object
 2   AIIMS   119 non-null    object
dtypes: int64(1), object(2)
memory usage: 3.5+ KB


In [129]:
data['RAMJA'].isnull().sum(axis=0)

3

In [130]:
ramjaNull = data['RAMJA'].isnull().sum(axis=0)
aiimsNull = data['AIIMS'].isnull().sum(axis=0)
ramjanonnull = TotalSamples - ramjaNull
aiimsnonnull = TotalSamples - aiimsNull

In [131]:
import re
def cleanup(text):
    text = str(text)
    text = text.upper()
    text = text.replace(".", " ")
    text = text.replace(" AND ", " ")
    text = re.sub(r'\s+', ' ', text)  # Use re.sub for regular expression replacement
    text = text.strip()
    return text


In [132]:
bacteriatypes = {
    "COLI" : 'Escherichia coli', 
    "PNEUMO" : 'Klebsiella pneumoniae',
    "AERUGI" : 'Pseudomonas aeruginosa',
    "BAUMAN" : 'Acinetobacter baumannii', 
    "MIRABIL" : 'Proteus mirabilis', 
    "FAECALI" : 'Enterococcus faecalis', 
    "AUREUS" : 'Staphylococcus aureus', 
    "SAPROPHY" : 'Staphylococcus saprophyticus',
    "STERI" : 'Sterile',
    "CONTA" : 'Contamination',
    "NAN" : 'NAN'
}

In [133]:
data['cleanRAMJA'] = data['RAMJA'].apply(cleanup)
data['cleanAIIMS'] = data['AIIMS'].apply(cleanup)

In [134]:
data.shape

(146, 5)

In [135]:
def check(mainstring, s1, s2):
    if (mainstring.count(s1) > 0 or mainstring.count(s2) > 0):
        #print("YES")
        return(1)
    else:
        #print("NO")
        return(0)

In [136]:
df_R = data.iloc[:,0:6]
genus = ["ESCHER","KLEBS","PSEUDO","ACINE","PROT","ENTER","AURE","SAPRO","STERI","CONTA","NAN"]
species = ["COLI", "PNEUMO", "AERUGI", "BAUMAN", "MIRABIL", "FAECALI", "AUREUS", "SAPROPHY", "STERI","CONTA","NAN"]
for s1, s2 in zip(genus,species):
    df_R[s2] = data['cleanRAMJA'].apply(lambda x: check(x, s1, s2))

df_R.rename(columns=bacteriatypes, inplace=True) #change column names
# add suffix _R to column names
# columns_to_modify = ['Escherichia coli','Klebsiella pneumoniae','Pseudomonas aeruginosa','Acinetobacter baumannii','Proteus mirabilis','Enterococcus faecalis','Staphylococcus aureus','Staphylococcus saprophyticus','Sterile','Contamination','NAN']
# data.rename(columns={col: f"{col}_R" for col in columns_to_modify}, inplace=True)

In [137]:
df_R.head()

,UHID,RAMJA,AIIMS,cleanRAMJA,cleanAIIMS,Escherichia coli,Klebsiella pneumoniae,Pseudomonas aeruginosa,Acinetobacter baumannii,Proteus mirabilis,Enterococcus faecalis,Staphylococcus aureus,Staphylococcus saprophyticus,Sterile,Contamination,NAN
0,20220065432,Enterococcus faecalis,Enterococcus faecalis,ENTEROCOCCUS FAECALIS,ENTEROCOCCUS FAECALIS,0,0,0,0,0,1,0,0,0,0,0
1,20240016182,NaN,Contamination,NAN,CONTAMINATION,0,0,0,0,0,0,0,0,0,0,1
2,20230125389,"E.Coli, K.pneumoniae, P. aeruginosa, A. bauman...",E.coli,"E COLI, K PNEUMONIAE, P AERUGINOSA, A BAUMANNI...",E COLI,1,1,1,1,0,0,1,0,0,0,0
3,20180022685,"K.pneumoniae, E. faecalis, P. mirabilis, A. b...",CONTAMINATION,"K PNEUMONIAE, E FAECALIS, P MIRABILIS, A BAUMA...",CONTAMINATION,0,1,0,1,1,1,1,0,0,0,0
4,20201415102,NaN,NaN,NAN,NAN,0,0,0,0,0,0,0,0,0,0,1


In [138]:
df_A = data.iloc[:,0:6]
genus = ["ESCHER","KLEBS","PSEUDO","ACINE","PROT","ENTER","AURE","SAPRO","STERI","CONTA","NAN"]
species = ["COLI", "PNEUMO", "AERUGI", "BAUMAN", "MIRABIL", "FAECALI", "AUREUS", "SAPROPHY", "STERI","CONTA","NAN"]
for s1, s2 in zip(genus,species):
    df_A[s2] = data['cleanAIIMS'].apply(lambda x: check(x, s1, s2))

df_A.rename(columns=bacteriatypes, inplace=True) #change column names

# add suffix _R to column names
#columns_to_modify = ['Escherichia coli','Klebsiella pneumoniae','Pseudomonas aeruginosa','Acinetobacter baumannii','Proteus mirabilis','Enterococcus faecalis','Staphylococcus aureus','Staphylococcus saprophyticus','Sterile','Contamination','NAN']
#data.rename(columns={col: f"{col}_A" for col in columns_to_modify}, inplace=True)

In [139]:
df_A.head()

,UHID,RAMJA,AIIMS,cleanRAMJA,cleanAIIMS,Escherichia coli,Klebsiella pneumoniae,Pseudomonas aeruginosa,Acinetobacter baumannii,Proteus mirabilis,Enterococcus faecalis,Staphylococcus aureus,Staphylococcus saprophyticus,Sterile,Contamination,NAN
0,20220065432,Enterococcus faecalis,Enterococcus faecalis,ENTEROCOCCUS FAECALIS,ENTEROCOCCUS FAECALIS,0,0,0,0,0,1,0,0,0,0,0
1,20240016182,NaN,Contamination,NAN,CONTAMINATION,0,0,0,0,0,0,0,0,0,1,0
2,20230125389,"E.Coli, K.pneumoniae, P. aeruginosa, A. bauman...",E.coli,"E COLI, K PNEUMONIAE, P AERUGINOSA, A BAUMANNI...",E COLI,1,0,0,0,0,0,0,0,0,0,0
3,20180022685,"K.pneumoniae, E. faecalis, P. mirabilis, A. b...",CONTAMINATION,"K PNEUMONIAE, E FAECALIS, P MIRABILIS, A BAUMA...",CONTAMINATION,0,0,0,0,0,0,0,0,0,1,0
4,20201415102,NaN,NaN,NAN,NAN,0,0,0,0,0,0,0,0,0,0,1


In [161]:
ramja = ['Escherichia coli_R','Klebsiella pneumoniae_R','Pseudomonas aeruginosa_R','Acinetobacter baumannii_R','Proteus mirabilis_R','Enterococcus faecalis_R','Staphylococcus aureus_R','Staphylococcus saprophyticus_R','Sterile_R','Contamination_R','NAN_R']
AIIMS = ['Escherichia coli_A','Klebsiella pneumoniae_R','Pseudomonas aeruginosa_R','Acinetobacter baumannii_R','Proteus mirabilis_R','Enterococcus faecalis_R','Staphylococcus aureus_R','Staphylococcus saprophyticus_R','Sterile_R','Contamination_R','NAN_R']

In [134]:
#data.to_excel("test1.xlsx")

In [99]:
df_R_nonnull = df_R.dropna(subset=['RAMJA','AIIMS'])
df_R_nonnull.shape

(118, 16)

In [100]:
df_A_nonnull = df_A.dropna(subset=['RAMJA','AIIMS'])
df_A_nonnull.shape

(118, 16)

In [187]:
#datanonnull.to_excel('test2.xlsx')

In [101]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(df_A_nonnull['Escherichia coli'],df_R_nonnull['Escherichia coli'])
cm


array([[68, 32],
       [ 7, 11]], dtype=int64)

In [102]:
print(df_R_nonnull['Escherichia coli'].value_counts())

0    75
1    43
Name: Escherichia coli, dtype: int64


In [103]:
conta_aiims = df_A['Contamination'].value_counts()[1]
conta_ramja = df_R['Contamination'].value_counts()[0]


In [107]:
from sklearn.metrics import confusion_matrix

y_true = df_A_nonnull['Escherichia coli']
y_pred = df_R_nonnull['Escherichia coli']

cm = confusion_matrix(y_true, y_pred)
print(cm)

[[68 32]
 [ 7 11]]


In [108]:
common_nonnull = df_R_nonnull.shape[0]

In [109]:
df_R_nonnull.head()

,UHID,RAMJA,AIIMS,cleanRAMJA,cleanAIIMS,Escherichia coli,Klebsiella pneumoniae,Pseudomonas aeruginosa,Acinetobacter baumannii,Proteus mirabilis,Enterococcus faecalis,Staphylococcus aureus,Staphylococcus saprophyticus,Sterile,Contamination,NAN
0,20220065432,Enterococcus faecalis,Enterococcus faecalis,ENTEROCOCCUS FAECALIS,ENTEROCOCCUS FAECALIS,0,0,0,0,0,1,0,0,0,0,0
2,20230125389,"E.Coli, K.pneumoniae, P. aeruginosa, A. bauman...",E.coli,"E COLI, K PNEUMONIAE, P AERUGINOSA, A BAUMANNI...",E COLI,1,1,1,1,0,0,1,0,0,0,0
3,20180022685,"K.pneumoniae, E. faecalis, P. mirabilis, A. b...",CONTAMINATION,"K PNEUMONIAE, E FAECALIS, P MIRABILIS, A BAUMA...",CONTAMINATION,0,1,0,1,1,1,1,0,0,0,0
5,20240012395,"Polymicrobial (E.coli, A. baumannii, P.mirabil...",E.coli,"POLYMICROBIAL (E COLI, A BAUMANNII, P MIRABILI...",E COLI,1,0,0,1,1,1,1,0,0,0,0
6,20220166372,"Pseudomonas, P.mirabilis, E.Faecalis, S. aureus",E.coli & Pseudomnas,"PSEUDOMONAS, P MIRABILIS, E FAECALIS, S AUREUS",E COLI & PSEUDOMNAS,0,0,1,0,1,1,1,0,0,0,0


In [142]:
df_R.shape

(146, 16)

In [143]:
df_R.head()

,UHID,RAMJA,AIIMS,cleanRAMJA,cleanAIIMS,Escherichia coli,Klebsiella pneumoniae,Pseudomonas aeruginosa,Acinetobacter baumannii,Proteus mirabilis,Enterococcus faecalis,Staphylococcus aureus,Staphylococcus saprophyticus,Sterile,Contamination,NAN
0,20220065432,Enterococcus faecalis,Enterococcus faecalis,ENTEROCOCCUS FAECALIS,ENTEROCOCCUS FAECALIS,0,0,0,0,0,1,0,0,0,0,0
1,20240016182,NaN,Contamination,NAN,CONTAMINATION,0,0,0,0,0,0,0,0,0,0,1
2,20230125389,"E.Coli, K.pneumoniae, P. aeruginosa, A. bauman...",E.coli,"E COLI, K PNEUMONIAE, P AERUGINOSA, A BAUMANNI...",E COLI,1,1,1,1,0,0,1,0,0,0,0
3,20180022685,"K.pneumoniae, E. faecalis, P. mirabilis, A. b...",CONTAMINATION,"K PNEUMONIAE, E FAECALIS, P MIRABILIS, A BAUMA...",CONTAMINATION,0,1,0,1,1,1,1,0,0,0,0
4,20201415102,NaN,NaN,NAN,NAN,0,0,0,0,0,0,0,0,0,0,1


In [171]:
df_R_m  = df_R.drop(columns= ['UHID','RAMJA', 'AIIMS', 'cleanRAMJA', 'cleanAIIMS','Contamination','Sterile','NAN'])
df_A_m  = df_A.drop(columns= ['UHID','RAMJA', 'AIIMS', 'cleanRAMJA', 'cleanAIIMS','Contamination','Sterile','NAN'])


In [173]:
df_R_m >= 1

,Escherichia coli,Klebsiella pneumoniae,Pseudomonas aeruginosa,Acinetobacter baumannii,Proteus mirabilis,Enterococcus faecalis,Staphylococcus aureus,Staphylococcus saprophyticus,Infection_R
0,False,False,False,False,False,True,False,False,True
1,False,False,False,False,False,False,False,False,True
2,True,True,True,True,False,False,True,False,True
3,False,True,False,True,True,True,True,False,True
4,False,False,False,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...
141,False,False,False,False,False,False,False,False,True
142,False,False,False,False,False,False,False,False,True
143,False,False,False,False,False,False,False,False,True
144,False,False,False,False,False,False,False,True,True


In [172]:
df_R_m >= 1

,Escherichia coli,Klebsiella pneumoniae,Pseudomonas aeruginosa,Acinetobacter baumannii,Proteus mirabilis,Enterococcus faecalis,Staphylococcus aureus,Staphylococcus saprophyticus,Infection_R
0,False,False,False,False,False,True,False,False,True
1,False,False,False,False,False,False,False,False,True
2,True,True,True,True,False,False,True,False,True
3,False,True,False,True,True,True,True,False,True
4,False,False,False,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...
141,False,False,False,False,False,False,False,False,True
142,False,False,False,False,False,False,False,False,True
143,False,False,False,False,False,False,False,False,True
144,False,False,False,False,False,False,False,True,True


In [168]:
df_R_m.head()

AttributeError: 'int' object has no attribute 'head'

In [160]:
df_R_m['Infection_R'] = df_R_m.any(axis=1)


,Escherichia coli,Klebsiella pneumoniae,Pseudomonas aeruginosa,Acinetobacter baumannii,Proteus mirabilis,Enterococcus faecalis,Staphylococcus aureus,Staphylococcus saprophyticus,Infection_R
0,0,0,0,0,0,1,0,0,1
1,0,0,0,0,0,0,0,0,1
2,1,1,1,1,0,0,1,0,1
3,0,1,0,1,1,1,1,0,1
4,0,0,0,0,0,0,0,0,1


In [161]:
df_R['Infection_R'] = df_R_m['Infection_R']
df_R.head()

,UHID,RAMJA,AIIMS,cleanRAMJA,cleanAIIMS,Escherichia coli,Klebsiella pneumoniae,Pseudomonas aeruginosa,Acinetobacter baumannii,Proteus mirabilis,Enterococcus faecalis,Staphylococcus aureus,Staphylococcus saprophyticus,Sterile,Contamination,NAN,Infection_R
0,20220065432,Enterococcus faecalis,Enterococcus faecalis,ENTEROCOCCUS FAECALIS,ENTEROCOCCUS FAECALIS,0,0,0,0,0,1,0,0,0,0,0,1
1,20240016182,NaN,Contamination,NAN,CONTAMINATION,0,0,0,0,0,0,0,0,0,0,1,1
2,20230125389,"E.Coli, K.pneumoniae, P. aeruginosa, A. bauman...",E.coli,"E COLI, K PNEUMONIAE, P AERUGINOSA, A BAUMANNI...",E COLI,1,1,1,1,0,0,1,0,0,0,0,1
3,20180022685,"K.pneumoniae, E. faecalis, P. mirabilis, A. b...",CONTAMINATION,"K PNEUMONIAE, E FAECALIS, P MIRABILIS, A BAUMA...",CONTAMINATION,0,1,0,1,1,1,1,0,0,0,0,1
4,20201415102,NaN,NaN,NAN,NAN,0,0,0,0,0,0,0,0,0,0,1,1


In [166]:
df_A_m['Infection_A'] = df_A_m.any(axis=1).astype(int)
df_A_m.head()

,Escherichia coli,Klebsiella pneumoniae,Pseudomonas aeruginosa,Acinetobacter baumannii,Proteus mirabilis,Enterococcus faecalis,Staphylococcus aureus,Staphylococcus saprophyticus,Infection_A
0,0,0,0,0,0,1,0,0,1
1,0,0,0,0,0,0,0,0,1
2,1,0,0,0,0,0,0,0,1
3,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0,1


,Escherichia coli,Klebsiella pneumoniae,Pseudomonas aeruginosa,Acinetobacter baumannii,Proteus mirabilis,Enterococcus faecalis,Staphylococcus aureus,Staphylococcus saprophyticus,Infection_A
0,0,0,0,0,0,1,0,0,1
1,0,0,0,0,0,0,0,0,1
2,1,0,0,0,0,0,0,0,1
3,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0,1


In [141]:
df_R_nonnull_m  = df_R_nonnull.drop(columns= ['UHID','RAMJA', 'AIIMS', 'cleanRAMJA', 'cleanAIIMS','Contamination','Sterile','NAN'])
df_A_nonnull_m  = df_A_nonnull.drop(columns= ['UHID','RAMJA', 'AIIMS', 'cleanRAMJA', 'cleanAIIMS','Contamination','Sterile','NAN'])
df_C_nonnull = df_A_nonnull_m + df_R_nonnull_m
df_C_nonnull.head()

,Acinetobacter baumannii,Enterococcus faecalis,Escherichia coli,Infection,Klebsiella pneumoniae,Proteus mirabilis,Pseudomonas aeruginosa,Staphylococcus aureus,Staphylococcus saprophyticus
0,0,2,0,NaN,0,0,0,0,0
2,1,0,2,NaN,1,0,1,1,0
3,1,1,0,NaN,1,1,0,1,0
5,1,1,2,NaN,0,1,0,1,0
6,0,1,1,NaN,0,1,2,1,0


In [116]:
df_C_nonnull = df_C_nonnull >= 2
df_C_nonnull.head()

In [118]:
df_C_nonnull['Infection'] = df_C_nonnull.any(axis=1)
df_C_nonnull

In [123]:
df_R_nonnull['Infection'] = df_C_nonnull['Infection'].astype(int)
df_R_nonnull.head()

C:\Users\neelesh\AppData\Local\Temp\ipykernel_15060\376528041.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_R_nonnull['Infection'] = df_C_nonnull['Infection'].astype(int)


In [272]:
""" import pandas as pd
from sklearn.metrics import confusion_matrix

# Lists of columns for comparison
ramja = ['Escherichia coli_R','Klebsiella pneumoniae_R','Pseudomonas aeruginosa_R','Acinetobacter baumannii_R','Proteus mirabilis_R','Enterococcus faecalis_R','Staphylococcus aureus_R','Staphylococcus saprophyticus_R','Sterile_R']
aiims = ['Escherichia coli_A','Klebsiella pneumoniae_A','Pseudomonas aeruginosa_A','Acinetobacter baumannii_A','Proteus mirabilis_A','Enterococcus faecalis_A','Staphylococcus aureus_A','Staphylococcus saprophyticus_A','Sterile_A']

# List to hold each confusion matrix as a DataFrame
cm_dfs = []

# Loop through each pair of columns and compute the confusion matrix
for s1, s2 in zip(aiims, ramja):
    cm = confusion_matrix(datanonnull[s1], datanonnull[s2])
    cm_dfs.append(cm.reshape(-1,1))

# Concatenate all confusion matrix DataFrames into one
#final_cm_df = pd.concat(cm_dfs)
cm_dfs = np.hstack(cm_dfs)
# Display the final concatenated DataFrame
print(cm_dfs) """


" import pandas as pd\nfrom sklearn.metrics import confusion_matrix\n\n# Lists of columns for comparison\nramja = ['Escherichia coli_R','Klebsiella pneumoniae_R','Pseudomonas aeruginosa_R','Acinetobacter baumannii_R','Proteus mirabilis_R','Enterococcus faecalis_R','Staphylococcus aureus_R','Staphylococcus saprophyticus_R','Sterile_R']\naiims = ['Escherichia coli_A','Klebsiella pneumoniae_A','Pseudomonas aeruginosa_A','Acinetobacter baumannii_A','Proteus mirabilis_A','Enterococcus faecalis_A','Staphylococcus aureus_A','Staphylococcus saprophyticus_A','Sterile_A']\n\n# List to hold each confusion matrix as a DataFrame\ncm_dfs = []\n\n# Loop through each pair of columns and compute the confusion matrix\nfor s1, s2 in zip(aiims, ramja):\n    cm = confusion_matrix(datanonnull[s1], datanonnull[s2])\n    cm_dfs.append(cm.reshape(-1,1))\n\n# Concatenate all confusion matrix DataFrames into one\n#final_cm_df = pd.concat(cm_dfs)\ncm_dfs = np.hstack(cm_dfs)\n# Display the final concatenated DataF

In [205]:
index = ['TN', 'FP', 'FN', 'TP']
columns = ['Escherichia coli','Klebsiella pneumoniae','Pseudomonas aeruginosa','Acinetobacter baumannii','Proteus mirabilis','Enterococcus faecalis','Staphylococcus aureus','Staphylococcus saprophyticus','Sterile']
df_cm = pd.DataFrame(cm_dfs,index,columns)
df_cm

,Escherichia coli,Klebsiella pneumoniae,Pseudomonas aeruginosa,Acinetobacter baumannii,Proteus mirabilis,Enterococcus faecalis,Staphylococcus aureus,Staphylococcus saprophyticus,Sterile
TN,68,88,103,93,91,88,90,107,42
FP,32,24,13,25,26,24,27,11,12
FN,7,4,0,0,0,3,1,0,38
TP,11,2,2,0,1,3,0,0,26


In [206]:
totalPositive = df_cm.loc['TP'] + df_cm.loc['FP']
totalNegative = df_cm.loc['TN'] + df_cm.loc['FN']
df_cm.loc['Total Positive'] = totalPositive
df_cm.loc['Total Negative'] = totalNegative

In [207]:
df_cm

,Escherichia coli,Klebsiella pneumoniae,Pseudomonas aeruginosa,Acinetobacter baumannii,Proteus mirabilis,Enterococcus faecalis,Staphylococcus aureus,Staphylococcus saprophyticus,Sterile
TN,68,88,103,93,91,88,90,107,42
FP,32,24,13,25,26,24,27,11,12
FN,7,4,0,0,0,3,1,0,38
TP,11,2,2,0,1,3,0,0,26
Total Positive,43,26,15,25,27,27,27,11,38
Total Negative,75,92,103,93,91,91,91,107,80


In [211]:
aiims2 = [TotalSamples, aiimsNull, aiimsnonnull, common_nonnull, contaaiims]

In [212]:
aiims2

[146, 27, 119, 118, 14]